# RAG Evaluation Without Ground Truth — Experiments

**Mục tiêu:** Thử nghiệm các framework evaluation không cần `expected_answer` và `ground_truth_context`

## Vấn đề

Hệ thống hiện tại phụ thuộc vào:
- ❌ `expected_answer` — không có sẵn trong thực tế
- ❌ `ground_truth_context` — không có vì RAG tự động retrieve

## Giải pháp

Test 3 frameworks:
1. **DeepEval** — LLM-as-Judge (no ground truth mode)
2. **RAGAS** — RAG-specific metrics
3. **OpenRAG-Eval** (Optional) — Research approach

## Workflow

1. Load test cases
2. Generate answers from RAG API
3. Evaluate with each framework
4. Meta-evaluate: So sánh consistency
5. Human-in-the-loop validation

---

## 📦 Setup & Installation

Cài đặt các thư viện cần thiết

In [1]:
!pip install ragas datasets langchain langchain-community langchain-openai
!pip install deepeval pandas matplotlib seaborn scipy

In [2]:
# Install packages nếu cần (uncomment)
# !pip install ragas datasets langchain langchain-community langchain-openai
# !pip install deepeval pandas matplotlib seaborn scipy

import os, sys, warnings, json, pandas as pd, numpy as np
from typing import Dict, List, Any
from dotenv import load_dotenv

# Suppress warnings
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.append(os.path.abspath('.'))

# Load .env và kiểm tra API key
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Bạn chưa đặt OPENAI_API_KEY trong file .env")

print("✅ Setup complete!")

✅ Setup complete!


## 1️⃣ Load Test Cases

Load và explore test cases từ `data/testcases.json`

In [3]:
# Load test cases
with open('../data/testcase/factual_testcase/results.json', 'r', encoding='utf-8-sig') as f:
    results = json.load(f)

print(f"📊 Loaded {len(results)} test cases\n")

# Display first test case
print("Example test case:")
for  testcase in results[:2]:
    print(f"ID: {testcase['question_id']}")
    print(f"Question: {testcase['question']}")



📊 Loaded 18 test cases

Example test case:
ID: F001
Question: Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu tín chỉ để tốt nghiệp?
ID: F002
Question: Thời gian đào tạo ngành CNTT kéo dài bao nhiêu năm?


## 3️⃣ RAGAS Evaluation (No Ground Truth)

RAGAS có các metrics không cần `expected_answer`:
- **Faithfulness** — Answer có faithful với retrieved context không?
- **Answer Relevancy** — Answer có relevant với question không?

Cả 2 metrics này đều reference-free!

In [4]:
! pip install langchain_openai


In [5]:
# Setup RAGAS with Ollama (local LLM)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Initialize Ollama LLM for RAGAS
ollama_llm = ChatOpenAI(model="gpt-4o-mini")
ollama_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

ragas_llm = LangchainLLMWrapper(ollama_llm)
ragas_embeddings = LangchainEmbeddingsWrapper(ollama_embeddings)

In [6]:
import json
from ragas import evaluate
from datasets import Dataset

file_path = "../data/testcase/factual_testcase/results.json"

with open(file_path, "r", encoding="utf-8-sig") as f:
    results = json.load(f)

# Lọc những trường hợp có answer hợp lệ
valid_results = [r for r in results if r.get("answer")]

# Chuẩn hoá contexts: RAGAS yêu cầu contexts là list[str], nhưng giữ cả title
def normalize_contexts(ctx_list):
    if not ctx_list:
        return []
    # Trả về list[str] theo format [title:{title}|content:{content}]
    return [f"[title:{c.get('title','')}|content:{c.get('content','')}]" for c in ctx_list]


ragas_data = {
    "question": [r["question"] for r in valid_results],
    "answer": [r["answer"] for r in valid_results],
    "contexts": [normalize_contexts(r["contexts"]) for r in valid_results],
    "ground_truth": [r.get("ground_truth", "") for r in valid_results],  # 🔥 thêm ground truth
}

ragas_dataset = Dataset.from_dict(ragas_data)


# In theo từng câu hỏi kèm content, answer và ground_truth
for i, r in enumerate(valid_results):
    print(f"Câu hỏi: {r['question']}\n")
    
    print("Nội dung:")
    for c in normalize_contexts(r["contexts"]):
        print(f"- {c}\n")  # xuống dòng sau mỗi content
    
    print(f"Answer: {r['answer']}\n")
    print(f"Ground truth: {r.get('ground_truth', '')}\n")
    
    print("="*50 + "\n")  # phân cách giữa các câu hỏi
    

Câu hỏi: Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu tín chỉ để tốt nghiệp?

Nội dung:
- [title:Hệ Thông Thông Tin - Chương Trình Tiên Tiến - 4\. Quy định đào tạo, điều kiện tốt nghiệp - Điều kiện tốt nghiệp|content:\- Công nhận tốt nghiệp: Sinh viên đã tích lũy tối thiểu là 130 tín chỉ, đã hoàn thành các môn học bắt buộc đối với ngành Hệ thống thông tin Chương trình đào tạo tiên tiến, trình độ Anh văn đạt yêu cầu theo quy định của Trường dành riêng cho chương trình tiên tiến.
\- Sinh viên phải đáp ứng đủ các tiêu chuẩn khác theo Quy chế đào tạo hiện hành.
\- Sinh viên tốt nghiệp được cấp bằng: Cử nhân Hệ thống thông tin – chương trình đào tạo tiên tiến.]

- [title:Trí Tuệ Nhân Tạo - 5\. CHƯƠNG TRÌNH ĐÀO TẠO - 5.4 Khối kiến thức tốt nghiệp|content:● Sinh viên tích lũy tối thiểu 10 tín chỉ cho khối kiến thức tốt nghiệp.
● Sinh viên được chọn:
○ Hoặc, thực hiện Khóa luận tốt nghiệp (KLTN) 10 tín chỉ nếu sinh viên đủ điều kiện làm khóa luận tốt nghiệp theo qui chế của trường.
○ Hoặ

In [7]:
# ---------------------------------------------------
# 2️⃣ Import metrics async
# ---------------------------------------------------

from ragas.metrics import AnswerCorrectness, AnswerRelevancy, Faithfulness, ContextRelevance, ContextRecall
import pandas as pd

metrics_dict = {
    "AnswerCorrectness": AnswerCorrectness(llm=ragas_llm),
    "AnswerRelevancy": AnswerRelevancy(embeddings=ragas_embeddings),
    "Faithfulness": Faithfulness(llm=ragas_llm),
    "ContextRelevance": ContextRelevance(llm=ragas_llm),
    "ContextRecall": ContextRecall()
}



In [12]:
# ---------------------------------------------------
# 2️⃣ Import các metric async-compatible
# ---------------------------------------------------

from ragas.metrics import AnswerCorrectness, AnswerRelevancy, Faithfulness, ContextRelevance, ContextRecall
import pandas as pd

metrics_dict = {
    "AnswerCorrectness": AnswerCorrectness(llm=ragas_llm),
    "AnswerRelevancy": AnswerRelevancy(embeddings=ragas_embeddings),
    "Faithfulness": Faithfulness(llm=ragas_llm),
    "ContextRelevance": ContextRelevance(llm=ragas_llm),
    "ContextRecall": ContextRecall()
}

summary_scores = {}

for name, metric in metrics_dict.items():
    print(f"\n🚀 Running metric: {name} ...")
    
    result = evaluate(
        dataset=ragas_dataset,
        metrics=[metric],
        llm=ragas_llm,
        embeddings=ragas_embeddings,
        batch_size=5,
        show_progress=True
    )
    
    score = list(result._scores_dict.values())[0]
    summary_scores[name] = score

df_scores = pd.DataFrame([summary_scores])
print("\n✅ Summary scores preview:")
print(df_scores)



🚀 Running metric: AnswerCorrectness ...


Evaluating: 100%|██████████| 18/18 [03:53<00:00, 13.00s/it]



🚀 Running metric: AnswerRelevancy ...


Evaluating: 100%|██████████| 18/18 [01:40<00:00,  5.56s/it]



🚀 Running metric: Faithfulness ...


...


🚀 Running metric: ContextRelevance ...


Evaluating: 100%|██████████| 18/18 [00:41<00:00,  2.29s/it]



🚀 Running metric: ContextRecall ...


Evaluating: 100%|██████████| 18/18 [00:50<00:00,  2.83s/it]



✅ Summary scores preview:
                                   AnswerCorrectness  \
0  [0.5296319994975731, 0.08293431489069471, 0.19...   

                                     AnswerRelevancy  \
0  [0.31958526800349046, 0.0, 0.3757968320058209,...   

                                        Faithfulness  \
0  [0.8571428571428571, 0.0, 1.0, 0.0, 1.0, 0.666...   

                                    ContextRelevance  \
0  [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.25, 1.0, 0.75...   

                                       ContextRecall  
0  [1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...  


In [13]:
import numpy as np

# Tạo dict để lưu trung bình
avg_scores = {}

for col in df_scores.columns:
    avg_scores[col] = np.mean(df_scores[col][0])  # df_scores[col][0] là list score

# Xuất kết quả
print("\n✅ Average score for each metric:")
for metric, avg in avg_scores.items():
    print(f"{metric}: {avg:.4f}")



✅ Average score for each metric:
AnswerCorrectness: 0.2388
AnswerRelevancy: 0.1244
Faithfulness: 0.5142
ContextRelevance: 0.6528
ContextRecall: 0.5370
